In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import time
import torch
import pandas as pd
import random
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


In [ ]:


# 모델과 토크나이저를 명시적으로 로드
model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# 파이프라인 생성 및 설정
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device="cuda"
)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:

# 입력 텍스트 설정
input_text = """p-축구M인천 ,qN 35c운드 r&Q{영`
위 문장은 축구 관련 기사 제목을 노이징한거야. 원래 제목을  추론해줘.
"""

# 텍스트 생성 시 세부 옵션 지정
output = pipe(
    input_text,
    max_length=100,          # 생성할 텍스트의 최대 길이
    temperature=0.7,        # 다양성 조절 (0.7은 적절한 온도)
    top_k=50,               # 상위 50개의 후보에서 선택
    top_p=0.9,              # 상위 90% 누적 확률에서 선택
    num_return_sequences=1,  # 반환할 응답의 수
    truncation=True  # 텍스트가 max_length를 넘으면 잘라냄
)

print(output)


[{'generated_text': 'p-축구M인천 ,qN 35c운드 r&Q{영`\n위 문장은 축구 관련 기사 제목을 노이징한거야. 원래 제목을  추론해줘.\n\n문장: "M인천, 35라운드에서 승리를 거두며 상위권 진입을 노리고 있다."\n\n추론된 원래 제목:\n"M인천, 35라운드에서 승리하며 상위권 진입 희망"\n\n설명'}]


LangChain에서 Hugging Face 모델을 연결하기 위해, 먼저 Transformers 라이브러리로 모델을 로드한 뒤 pipeline 객체를 생성합니다.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the tokenizer and model
model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)


# GPU 사용 가능 시 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
# Load few-shot-example and need_denoise data
few_shot_df = pd.read_csv("../../data/5_base_noise_detected_test_with_predictions.csv")  # ID, text, target columns
need_denoise_df = pd.read_csv("../../data/5_base_noise_detected_train.csv")  # ID, text, target columns with noise

In [7]:
# Split few_shot_df into 7 dataframes based on target labels (0 to 6)
label_dfs = {label: few_shot_df[few_shot_df['target'] == label] for label in range(7)}


In [24]:
def add_extended_ascii_noise(text, noise_level=0.5):
    noisy_text = ""
    for char in text:
        if random.random() < noise_level:
            # ASCII와 확장된 범위에서 임의의 문자 선택
            if random.random() < 0.93:
                # ASCII 범위 내 특수 문자, 대소문자, 숫자
                noisy_text += chr(random.randint(33, 126))  
            else:
                # 한자 유니코드 범위 (CJK 범위 일부 예시)
                noisy_text += chr(random.randint(0x4E00, 0x9FFF))  
        else:
            noisy_text += char
    return noisy_text

In [23]:
# 예시: label 데이터 5개 뽑기 및 노이즈 추가
few_shot_examples = label_dfs[4].sample(5)
sample_list = "예시: \n"
for _, example_row in few_shot_examples.iterrows():
    # 아스키 기반 노이즈 추가
    original_example = example_row['text']
    noisy_example = add_extended_ascii_noise(original_example)
    sample_list += f"입력된 제목: {noisy_example}\n복원된 제목: {original_example}\n\n"

print(sample_list)

예시: 
입력된 제목: UrE }텔 垎4f학/술f진I회 대Li6沍j2
복원된 제목: 게시판 인텔 국제과학기술경진대회 대표단 발대식

입력된 제목: &아F 드]"빙 시뮬레V6
복원된 제목: 세아트 드라이빙 시뮬레이터

입력된 제목: 1r∼(u대는1유aX F대…하FF4p4n 9E< 본吔
복원된 제목: 10∼20대는 유튜브 세대…하루 4.4회 52분 본다

입력된 제목: -K. 미7d,객 잡5다
복원된 제목: SKT 미래 고객 잡는다

입력된 제목: '탁J鏊%訛D워P그룹과C분~^부X기@y공동p구泐纚B
복원된 제목: 예탁원 CSD워킹그룹과 분산장부 기술 공동연구 협약




In [27]:
# Function to generate denoised text for each entry in need_denoise_df
def denoise_text_with_few_shot(row, label_dfs):
    target_label = row['target']
    noisy_text = row['text']
    
    # 같은 label 데이터 3개를 뽑아서 few-shot으로 활용
    few_shot_examples = label_dfs[target_label].sample(3)
    sample_list = "예시: \n"
    for _, example_row in few_shot_examples.iterrows():
        # 아스키 기반 노이즈 추가
        original_example = example_row['text']
        noisy_example = add_extended_ascii_noise(original_example)
        sample_list += f"입력된 제목: {noisy_example}\n복원된 제목: {original_example}\n\n"
    
    # Construct few-shot prompt
    prompt = """다음은 아스키코드가 치환되는 방식으로 많은 노이즈가 있는 한국어 뉴스 기사 제목입니다.
    노이즈가 들어가지 않은 한글부분을 참고하여 원래의 정확한 한국어 제목으로 복원해주세요.
    앞뒤에 설명 붙이지 말고, 반드시 복원된 제목만 대답하세요.
    아래의 예시는 동일한 주제의 복원된 뉴스기사 제목입니다.
    예시를 참고하여 예시와 반드시 동일한 주제로 복원하세요.
    입력된 제목보다 짧지 않도록 생성하세요.
    """
    prompt += sample_list
    prompt += f"""입력된 제목: {noisy_text}
    복원된 제목:"""

    # Generate denoised text using the model
    
    # # pad_token이 없으면 eos_token을 pad_token으로 설정
    # if tokenizer.pad_token is None:
    #     tokenizer.pad_token = tokenizer.eos_token
    
    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True).to(device) # 기본
    # inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512) # 라마
    # input_ids = inputs["input_ids"].to(device)
    # attention_mask = inputs["attention_mask"].to(device)
    outputs = model.generate(
            inputs, # 라마제외
            # input_ids, # 라마
            max_new_tokens=20,            
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.9,
            temperature=0.7,
            do_sample=True,
            # attention_mask=attention_mask,              # Attention mask 추가 (라마)
            # pad_token_id=tokenizer.eos_token_id,        # pad_token_id를 eos_token_id로 설정 (라마)
                             )
    restored_headline = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # "복원된 제목:" 이후의 텍스트 추출
    if "복원된 제목:" in restored_headline:
        restored_headline = restored_headline.split("복원된 제목:")[-1].strip()
    # "답변:" 부분 제거
    restored_headline = re.sub(r"^답변:\s*", "", restored_headline)
    
    # 줄바꿈 이후 내용 자르기, 예외 발생 시 빈 문자열 할당
    try:
        restored_headline = restored_headline.splitlines()[0]
    except IndexError:
        restored_headline = ""

    
    
    print(f"\n{sample_list}")
    print("--------------------------------------------")
    print(f"결과 : {noisy_text} -- {restored_headline}")
    print("--------------------------------------------")

    return restored_headline.strip()

In [28]:
import time
import os
import pandas as pd

# 기존 파일이 있는지 확인하고 불러오기
save_path = "denoised_results.csv"

if os.path.exists(save_path):
    # 기존 진행 파일 불러오기
    need_denoise_df = pd.read_csv(save_path)
    # 'denoised_text' 열에 NaN 값만 남아있는 인덱스를 찾음
    last_processed_index = need_denoise_df['denoised_text'].last_valid_index() or -1
else:
    # 파일이 없으면 처음부터 시작
    last_processed_index = -1
    need_denoise_df['denoised_text'] = ""

total_rows = len(need_denoise_df)
print(f"Resuming from row {last_processed_index + 1}...")

# 각 행을 순회하며 denoise 함수 적용 및 중간 진행 저장
for index, row in need_denoise_df.iterrows():
    if index <= last_processed_index:
        continue  # 이미 처리한 행은 건너뜀
    
    start_time = time.time()
    
    try:
        # 현재 행에 대한 denoised_text 생성
        denoised_text = denoise_text_with_few_shot(row, label_dfs)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        denoised_text = ""  # 오류 시 빈 문자열로 대체
    
    # denoised_texts 리스트에 추가
    need_denoise_df.at[index, 'denoised_text'] = denoised_text
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Processed {index + 1}/{total_rows} rows - Time taken: {elapsed_time:.2f} seconds.")
    
    # 100행마다 중간 진행 저장
    if (index + 1) % 100 == 0:
        need_denoise_df.to_csv(save_path, index=False)
        print(f"Checkpoint saved at row {index + 1}.")

# 최종 저장
need_denoise_df.to_csv(save_path, index=False)
print("Final denoised results saved.")


Resuming from row 0...

예시: 
입력된 제목: 한국MSU윈鰍1t IoT T디dQ*시UB2B LC
복원된 제목: 한국MS 윈도10 IoT 에디션 출시…B2B 공략

입력된 제목: 네咼버 R문~& 검색 내6D안에 y글0櫻U"tY것
복원된 제목: 네이버 전문자료 검색 내년 안에 구글 따라잡을 것

입력된 제목: .글k/세대 "롬7스l 출$_C세Z%`?씨Q단W 0O종O
복원된 제목: 구글 3세대 크롬캐스트 출시…신세계아이앤씨 단독 판매종합


--------------------------------------------
결과 : 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보 -- 정조대왕 파초선 미사리에서 KT 이용 기간 12분 동안 공개
--------------------------------------------
Processed 1/1602 rows - Time taken: 17.81 seconds.

예시: 
입력된 제목: 직원莔?c에 j력 행gU}t!*塖1조'Z혁신센UG V유
복원된 제목: 직원 채용에 압력 행사한 전 대구창조경제혁신센터장 집유

입력된 제목: e천2공s유치원C교사 8{D Yy 중 질{(l어
복원된 제목: 인천 공립유치원 교사 87% 근무 중 질병 얻어

입력된 제목: 이통 K/점들6완Gp급제법Ax속 추V 유감E부n용hB
복원된 제목: 이통 판매점들 완전자급제법 졸속 추진 유감…부작용 커


--------------------------------------------
결과 : K찰.국DLwo 로L3한N% 회장 2 T0&}송= -- 한국당 대표 후보들로 구성된 한나라연합회 회장 송 후보
--------------------------------------------
Processed 2/1602 rows - Time taken: 17.23 seconds.

예시: 
입력된 제목: 靑M시o頑%ENI0烌으로@勂,#Z구라도z좋다
복원된 제목: 靑 시선은 벌써 특검으로…후보 누구라도 좋다

입력된 제목: 北0무~